In [2]:
import climt
from climt._core.initialization import HybridSigmaPressureDiagnosticComponent
from gfs_dynamical_core import GFSDynamicalCore
from sympl import (
    PlotFunctionMonitor, NetCDFMonitor, set_constant,
    TimeDifferencingWrapper, UpdateFrequencyWrapper,
    get_constant, reset_constants, DataArray, DiagnosticComponent,
    combine_component_properties
)
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import xarray as xr
from xarray import DataArray
import xesmf as xe
from scipy.interpolate import CubicSpline, interp1d
import pkg_resources

In [3]:
# Load the Dataset from the NetCDF file
loaded_era5_data = xr.open_dataset('/run/media/adamh/X6/test_levels/era5_regrid_0101180000_in_Pa.nc')
loaded_era5_interpolated_data = xr.open_dataset('/run/media/adamh/X6/test_levels/era5_interpolation_0101180000.nc')

# Access the DataArrays by their new names
divergence_of_wind_regrid = loaded_era5_data['divergence_of_wind_regrid']
geopotential_regrid = loaded_era5_data['geopotential_regrid']
specific_humidity_regrid = loaded_era5_data['specific_humidity_regrid']
air_temperature_regrid = loaded_era5_data['air_temperature_regrid']
eastward_wind_regrid = loaded_era5_data['eastward_wind_regrid']
northward_wind_regrid = loaded_era5_data['northward_wind_regrid']
relative_vorticity_regrid = loaded_era5_data['relative_vorticity_regrid']
dewpoint_temperature_regrid = loaded_era5_data['dewpoint_temperature_regrid']
surface_temperature_regrid = loaded_era5_data['surface_temperature_regrid']
surface_geopotential_regrid = loaded_era5_data['surface_geopotential_regrid']
surface_pressure_regrid = loaded_era5_data['surface_pressure_regrid']


divergence_of_wind_interpolated = loaded_era5_interpolated_data['divergence_of_wind_interpolated']
geopotential_interpolated = loaded_era5_interpolated_data['geopotential_interpolated']
specific_humidity_interpolated = loaded_era5_interpolated_data['specific_humidity_interpolated']
air_temperature_interpolated = loaded_era5_interpolated_data['air_temperature_interpolated']
eastward_wind_interpolated = loaded_era5_interpolated_data['eastward_wind_interpolated']
northward_wind_interpolated = loaded_era5_interpolated_data['northward_wind_interpolated']
relative_vorticity_interpolated = loaded_era5_interpolated_data['relative_vorticity_interpolated']
dewpoint_temperature_regrid = loaded_era5_data['dewpoint_temperature_regrid']
surface_temperature_regrid = loaded_era5_data['surface_temperature_regrid']
surface_geopotential_regrid = loaded_era5_data['surface_geopotential_regrid']
surface_pressure_regrid = loaded_era5_data['surface_pressure_regrid']

/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [4]:
def plot_function(fig, state):

    ax = fig.add_subplot(2, 2, 1)
    state['specific_humidity'].mean(
        dim='lon').plot.contourf(
            ax=ax, levels=16, robust=True)
    ax.set_title('Specific Humidity')

    ax = fig.add_subplot(2, 2, 3)
    state['eastward_wind'].mean(dim='lon').plot.contourf(
        ax=ax, levels=16, robust=True)
    ax.set_title('Eastward Wind')

    ax = fig.add_subplot(2, 2, 2)
    state['northward_wind'].transpose().mean(
        dim='lon').plot.contourf(
        ax=ax, levels=16, robust=True)
    ax.set_title('Northward Wind')

    ax = fig.add_subplot(2, 2, 4)
    state['air_temperature'].mean(dim='lon').plot.contourf(
        ax=ax, levels=16)
    ax.set_title('Temperature')

    fig.tight_layout()


fields_to_store = ['atmosphere_hybrid_sigma_pressure_a_coordinate_on_interface_levels',
                   'atmosphere_hybrid_sigma_pressure_b_coordinate_on_interface_levels',
                   'surface_air_pressure', 'time', 'air_pressure', 
                   'air_pressure_on_interface_levels', 'longitude', 'latitude',
                   'air_temperature', 'eastward_wind', 'northward_wind', 'divergence_of_wind',
                   'atmosphere_relative_vorticity', 'surface_geopotential', 'specific_humidity'
                  ]

# Create plotting object
monitor = PlotFunctionMonitor(plot_function)
netcdf_monitor = NetCDFMonitor('/run/media/adamh/X6/test_levels/net_cdf_monitor.nc',
                               write_on_store=True,
                               store_names=fields_to_store)

monitor = PlotFunctionMonitor(plot_function)

set_constant('reference_air_pressure', value=1e5, units='Pa')
set_constant('top_of_model_pressure', value=100, units='Pa')
dycore = GFSDynamicalCore()
dcmip = climt.DcmipInitialConditions(add_perturbation=True)

grid = climt.get_grid(nx=128, ny=64, nz=37,
                      n_ice_interface_levels=None,
                      p_surf_in_Pa=96809.03361996275, 
                      p_toa_in_Pa=100,
                      custom_surface_pressure=surface_pressure_regrid,
                      )        

my_state = climt.get_default_state([dycore], grid_state=grid)
timestep = timedelta(minutes=10)

# out = dcmip(my_state)
# my_state.update(out)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [5]:
my_state['air_temperature'].values = air_temperature_interpolated.values
my_state['eastward_wind'].values = eastward_wind_interpolated.values
my_state['northward_wind'].values = northward_wind_interpolated.values
my_state['divergence_of_wind'].values = divergence_of_wind_interpolated.values
my_state['atmosphere_relative_vorticity'].values = relative_vorticity_interpolated.values
my_state['surface_geopotential'].values = surface_geopotential_regrid.values
my_state['specific_humidity'].values = specific_humidity_interpolated.values
my_state.update()


for i in range(6):
    diag, my_state = dycore(my_state, timestep)
    my_state.update(diag)
    my_state['time'] += timestep

    netcdf_monitor.store(my_state)
    monitor.store(my_state)

    print(my_state['time'])

2000-01-01 00:10:00
2000-01-01 00:20:00
2000-01-01 00:30:00
2000-01-01 00:40:00
2000-01-01 00:50:00
2000-01-01 01:00:00


In [7]:
data = xr.open_dataset('/run/media/adamh/X6/test_levels/net_cdf_monitor.nc')
my_state_0, my_state_1 = data.isel(time=0), data.isel(time=1)
#  Calculate residuals
residual_surface_pressure = my_state_0['surface_air_pressure'].values/100 - surface_pressure_regrid.values/100
residual_air_temperature = my_state_0['air_temperature'].values - air_temperature_interpolated.values
residual_eastward_wind = my_state_0['eastward_wind'].values - eastward_wind_interpolated.values
residual_northward_wind = my_state_0['northward_wind'].values - northward_wind_interpolated.values
residual_divergence_wind = my_state_0['divergence_of_wind'].values - divergence_of_wind_interpolated.values
residual_relative_vorticity = my_state_0['atmosphere_relative_vorticity'].values - relative_vorticity_interpolated.values
residual_surface_geopotential = my_state_0['surface_geopotential'].values - surface_geopotential_regrid.values
residual_specific_humidity = my_state_0['specific_humidity'].values - specific_humidity_interpolated.values

def residual_info_print(residual_str, residual, p=None):
    if p==None:
        print(f"{residual_str:<35} {my_state[residual_str].units:<15} {residual.mean():<30} {residual.max():<30} {residual.min():<30}")
    else:
        print(f"{residual_str:<35} {'hPa':<15} {residual.mean():<30} {residual.max():<30} {residual.min():<30}")


print(f"{'Variable':<35} {'Units':<15} {'Mean':<30} {'Max':<30} {'Min':<30}")
print("="*140)

residual_info_print('surface_air_pressure', residual_surface_pressure, p=True)
residual_info_print('air_temperature', residual_air_temperature)
residual_info_print('eastward_wind', residual_eastward_wind)
residual_info_print('northward_wind', residual_northward_wind)
residual_info_print('divergence_of_wind', residual_divergence_wind)
residual_info_print('atmosphere_relative_vorticity', residual_relative_vorticity)
residual_info_print('surface_geopotential', residual_surface_geopotential)
residual_info_print('specific_humidity', residual_specific_humidity)

Variable                            Units           Mean                           Max                            Min                           
surface_air_pressure                hPa             14.12384933204468              154.17084239244377             -91.26702620116203            
air_temperature                     degK            -0.0014391751584193239         10.301554509456565             -9.487683075233008            
eastward_wind                       m s^-1          -0.008687656703169993          12.454588510661218             -12.136702574281294           
northward_wind                      m s^-1          -0.00016876655820842807        13.40795508715916              -13.023190823956243           
divergence_of_wind                  s^-1            1.878610042900903e-07          9.624579592697222e-05          -7.428192262158134e-05        
atmosphere_relative_vorticity       s^-1            4.2891307845304754e-08         0.0002495849632014036          -0.0001757807655

In [9]:
residual_surface_pressure = my_state_1['surface_air_pressure'].values/100 - my_state_0['surface_air_pressure'].values/100
residual_air_temperature = my_state_1['air_temperature'].values - my_state_0['air_temperature'].values
residual_eastward_wind = my_state_1['eastward_wind'].values - my_state_0['eastward_wind'].values
residual_northward_wind = my_state_1['northward_wind'].values - my_state_0['northward_wind'].values
residual_divergence_wind = my_state_1['divergence_of_wind'].values - my_state_0['divergence_of_wind'].values
residual_relative_vorticity = my_state_1['atmosphere_relative_vorticity'].values - my_state_0['atmosphere_relative_vorticity'].values
residual_surface_geopotential = my_state_1['surface_geopotential'].values - my_state_0['surface_geopotential'].values
residual_specific_humidity = my_state_1['specific_humidity'].values - my_state_0['specific_humidity'].values

print(f"{'Variable':<35} {'Units':<15} {'Mean':<30} {'Max':<30} {'Min':<30}")
print("="*140)

residual_info_print('surface_air_pressure', residual_surface_pressure, p=True)
residual_info_print('air_temperature', residual_air_temperature)
residual_info_print('eastward_wind', residual_eastward_wind)
residual_info_print('northward_wind', residual_northward_wind)
residual_info_print('divergence_of_wind', residual_divergence_wind)
residual_info_print('atmosphere_relative_vorticity', residual_relative_vorticity)
residual_info_print('surface_geopotential', residual_surface_geopotential)
residual_info_print('specific_humidity', residual_specific_humidity)

Variable                            Units           Mean                           Max                            Min                           
surface_air_pressure                hPa             -0.004080392024438252          2.938712093540744              -4.029788411650884            
air_temperature                     degK            -3.2973805197754146e-05        1.0976455653344885             -0.8353321687148423           
eastward_wind                       m s^-1          0.00014358211986278586         1.915107014379963              -1.7666249104452638           
northward_wind                      m s^-1          0.0009290242560810859          3.0584143712435923             -2.9327083312996702           
divergence_of_wind                  s^-1            -1.5052454089418422e-09        1.3317440014812667e-05         -1.2878826442497036e-05       
atmosphere_relative_vorticity       s^-1            3.6070417561848536e-09         6.519383644270384e-06          -6.3206973467443